# Third Model

In diesem Modell, sollen die Arousal, Valence als Parameter mit verwendet werden.
1. Teil Testetn wie die Modelle welche Accuracy sie haben
2. Teil Auswerten was sie sagen
3. Teil auf basis dessen modelle trainieren

In [1]:
#!pip install transformers

### Kuenstliche Daten erstellen

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
from tqdm import tqdm
import time

In [3]:
tokenizer_eomtion = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
tokenizer_Fake_Model = AutoTokenizer.from_pretrained('EmotionsProject/model_Clasic')
tokenizer_Aurosal = AutoTokenizer.from_pretrained('Ann/Arousal_Model/')
tokenizer_Valence = AutoTokenizer.from_pretrained('Ann/Valence_Model/')


In [4]:
model_evaluation= AutoModelForSequenceClassification.from_pretrained('SamLowe/roberta-base-go_emotions')
model_fake_model = AutoModelForSequenceClassification.from_pretrained('EmotionsProject/model_Clasic')
model_arousal = AutoModelForSequenceClassification.from_pretrained('Ann/Arousal_Model/')
model_valence = AutoModelForSequenceClassification.from_pretrained('Ann/Valence_Model/')

In [5]:
model_valence.eval()
model_arousal.eval()
model_evaluation.eval()
model_fake_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
def valence_decode_vectorized(series):
    valence_map = {0: 'positive', 1: 'neutral', 2: 'negative'}
    return valence_map[series]

def arousal_decode_vectorized(series):
    arousal_map = {0: 'high', 1: 'medium', 2: 'low'}
    return arousal_map[series]

In [3]:
df = pd.read_csv("../Dataset/train_emotion_together.csv")
#df = pd.read_csv("../Dataset/evaluate.csv")# both necessary

In [9]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'majority_target', 'statement',
       'BinaryNumTarget', 'tweet', 'followers_count', 'friends_count',
       'favourites_count', 'statuses_count', 'listed_count', 'following',
       'embeddings', 'BotScore', 'BotScoreBinary', 'cred',
       'normalize_influence', 'mentions', 'quotes', 'replies', 'retweets',
       'favourites', 'hashtags', 'URLs', 'unique_count', 'total_count',
       'ORG_percentage', 'NORP_percentage', 'GPE_percentage',
       'PERSON_percentage', 'MONEY_percentage', 'DATE_percentage',
       'CARDINAL_percentage', 'PERCENT_percentage', 'ORDINAL_percentage',
       'FAC_percentage', 'LAW_percentage', 'PRODUCT_percentage',
       'EVENT_percentage', 'TIME_percentage', 'LOC_percentage',
       'WORK_OF_ART_percentage', 'QUANTITY_percentage', 'LANGUAGE_percentage',
       'Word count', 'Max word length', 'Min word length',
       'Average word length', 'present_verbs', 'past_verbs', 'adjectives',
       'adverbs', 'adposition

In [8]:
def predict(text, model,tokenizer):
    # Tokenisieren Sie den Eingabetext
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Machen Sie eine Vorhersage
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Holen Sie sich die Logits
    logits = outputs.logits
    
    # Wenden Sie Softmax an, um Wahrscheinlichkeiten zu erhalten
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    # Holen Sie sich die vorhergesagte Klasse
    #predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    return  probabilities[0].tolist()


def evaluate(df):
    total = len(df)
    ls_arousal = []
    ls_valence = []
    ls_fake = []
    required_columns = [ 'Fake_pred', 'Fake_perc']
    for col in required_columns:
        if col not in df.columns:
            df[col] = None  
    for index, row in tqdm(df.iterrows(), total=total, desc="Processing Rows", unit="row"):
        predict_arousal, perc_arousal = predict(row['tweet'],model_arousal, tokenizer_Aurosal)
        predict_valence, perc_valence = predict(row['tweet'],model_valence, tokenizer_Valence)
        perc = predict(row['tweet'],model_evaluation, tokenizer_eomtion)
        predict_fake_model, perc_fake_model = predict(row['tweet'],model_fake_model, tokenizer_Fake_Model)
        predict_valence = valence_decode_vectorized(predict_valence)
        predict_arousal = arousal_decode_vectorized(predict_arousal)
        
        df.at[index, 'emotion_model'] = perc
        df.at[index, 'Arousal_perc'] = perc_arousal
        df.at[index, 'Valence_pred'] = predict_valence
        df.at[index, 'Valence_perc'] = perc_valence
        df.at[index, 'Fake_pred'] = predict_fake_model
        df.at[index, 'Fake_perc'] = perc_fake_model
    return df

In [10]:
df_labeld = evaluate(df)

Processing Rows: 100%|██████████| 21602/21602 [20:08<00:00, 17.87row/s]


In [11]:
#df_labeld.to_csv("../Dataset/labeld_all.csv")
#df_labeld.to_csv("../Dataset/evaluation_annotated.csv")